<a href="https://colab.research.google.com/github/lephuocdat2000/AI-Tempo-Run/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [26]:
cd /content/gdrive/MyDrive/AITempoRun

/content/gdrive/MyDrive/AITempoRun


In [21]:
def Fixed_Coordinate(bbox):
  bbox = np.array(bbox)
  bbox = np.reshape(bbox,(4,2))
  xmin,xmax = np.amin(bbox[:,0],axis = 0),np.amax(bbox[:,0],axis=0)
  ymin,ymax = np.amin(bbox[:,1],axis=0), np.amax(bbox[:,1],axis=0)
  return xmin,ymin,xmax,ymax

def IOU_Calculation(gt_bbox,training_bbox):
  xmin_gt,ymin_gt,xmax_gt,ymax_gt = Fixed_Coordinate(gt_bbox)
  xmin,ymin,xmax,ymax = Fixed_Coordinate(training_bbox)
  if (xmax_gt < xmin) or (ymax_gt<ymin) or (xmin_gt > xmax) or (ymin_gt>ymax): return 0.0
  GT_bbox_area = (xmax_gt - xmin_gt + 1 ) * (ymax_gt - ymin_gt + 1 )
  Pre_bbox_area = (xmax - xmin + 1 ) * (ymax - ymin + 1 )
  xmin = max(xmin_gt,xmin)
  ymin = max(ymin_gt,ymin)
  xmax = min(xmax_gt,xmax)
  ymax = min(ymax_gt,ymax)
  intersection_area = (xmax-xmin+1)*(ymax-ymin+1)
  union_area = GT_bbox_area + Pre_bbox_area - intersection_area
  return intersection_area/union_area

In [28]:
import os
import numpy as np
import cv2

dictionary = os.listdir('result')

for file in dictionary:
    if file.endswith(".txt"):
       f1 = open(os.path.join('result',file),"r").readlines()
       if len(f1)!=0:
          image = cv2.imread(os.path.join('result',file[0:-4],'.jpg'))
          f2 = open(os.path.join('train/labels',file[4:]),"r").readlines()
          f3 = open(os.path.join('Preprocessing Data/Labeling with CRAFT',file[4:],'.txt'),"a")
          mark = np.zeros((len(f1),))
          for i in range(len(f1)-1):
              coordinate1 = f1[i].split(",")
              coordinate1 = [int(i) for i in coordinate1]
              max, selection = 0.0,None
              for j in range(len(f2)-1):
                if mark[j]!=0:
                    fence = f2[j].rfind(",")
                    coordinate2 = f2[j][0:fence].split()
                    coordinate2 = [int(i) for i in coordinate2]
                    IoU = IOU_Calculation(coordinate1,coordinate2)
                    if IoU > 0.0: 
                        mark[j] = 1
                        if max<IoU : max,selection = IoU,coordinate2
              f3.write(selection)
          f3.close()
          f2.close()
          cv2.imwrite(os.path.join('Preprocessing Data/Labeling with CRAFT',file[4:],'.jpg'),image) 
       f1.close()                        
       break

FileNotFoundError: ignored